In [11]:
from parler.main import Parler
import pandas as pd
from tqdm import tqdm

### Top 20 hashtags

The following requires the user to enter a search term and returns the top 20 hashtags sorted by volume

In [14]:
q = input('Enter search term: ')

dict1 = {}
dict1['tag'] = []
dict1['totalPosts'] = []

print('Collecting hashtags that contain ' + q + '..')

fetch_hashtags = Parler().hashtags(q)

for x in fetch_hashtags['tags']:
    dict1['tag'].append(x['tag'])
    dict1['totalPosts'].append(x['totalPosts'])

df_tags = pd.DataFrame.from_dict(dict1)

df_tags

,tag,totalPosts
0,wpww,127


### Top 10 affiliate domains

The following fetches the domains for a specified amount of pages (in this case 5) of affiliate news posts. Just change the "5" to a higher number to increase the number of posts you want to collect. 

In [18]:
dict1 = {}
dict1["domain"] = []

fetch_affiliates = Parler().affiliates()
next_cursor = fetch_affiliates["next"]

# There are a lot of domains named "feedproxy.google.com", so I added a few lines to get the site if this is the domain
for x in fetch_affiliates["links"]:
    if x["domain"] != "feedproxy.google.com":
        dict1["domain"].append(x["domain"])
    elif x["domain"] == "feedproxy.google.com":
        dict1["domain"].append(x["metadata"]["site"])

# Change "10" to a higher number to collect more posts
for i in tqdm(range(1,10),desc="Collecting affiliate data..."):
    fetch_affiliates = Parler().affiliates(cursor=next_cursor)
    next_cursor = fetch_affiliates["next"]
    for x in fetch_affiliates["links"]:
        if x["domain"] != "feedproxy.google.com":
            dict1["domain"].append(x["domain"])
        elif x["domain"] == "feedproxy.google.com":
            dict1["domain"].append(x["metadata"]["site"])

df_affiliate_data = pd.DataFrame.from_dict(dict1)

# Return 10 most frequent domains
most_freq = pd.Series(' '.join(df_affiliate_data["domain"]).lower().split()).value_counts()[:10]
print(most_freq)

pagesix.com              20
bizpacreview.com         20
lacortenews.com          20
thepostmillennial.com    20
@zerohedge               20
bigleaguepolitics.com    10
rt.com                   10
waynedupree.com          10
ammoland.com             10
dtype: int64



### Retrieve all posts which contain a specified hashtag

The following pages through and collects all posts which contain a specified hashtag and puts the id of the post and body of the post into a data frame for manipulation later

In [23]:
hashtag = input('Enter hashtag: ')

dict1 = {}
dict1["id"] = []
dict1["body"] = []

df_hashtag_data = pd.DataFrame()
    
print("Collecting data for #" + hashtag + "...")
    
fetch = Parler().hashtag_feed(hashtag)
next_cursor = fetch["next"]
    
for x in fetch["posts"]:
    dict1["id"].append(x["id"])
    dict1["body"].append(x["body"])

while fetch["last"] == False:
    fetch = Parler().hashtag_feed(hashtag,cursor=next_cursor)
    next_cursor = fetch["next"]
    for x in fetch["posts"]:
        dict1["id"].append(x["id"])
        dict1["body"].append(x["body"])
    
df_hashtag_data = pd.DataFrame.from_dict(dict1)
df_hashtag_data

,id,body
0,00b157d3554e4b469f5224daa7f9dfda,It’s amazing to me the parallels in time that ...
1,8548fe7ff75249a8a9b517da13a87f49,It’s real. \n\n#theeternaljew #itsthejewsdummy...
2,f3263fff0b1d4b08b062f2eefdd23e09,500 followers. 500 seekers of truth. 500 sons ...
3,c297ac3802b8441bb65d04b564c30eff,The hypocrisy...the lies...the double standard...
4,f3a0d17aa4794653bfb03164399359c0,The fact that there is a group of satanic pedo...
...,...,...
113,c810ab3811d041aab4b23a94050a2fd7,#antifa #notracist #libtards #blacklivesmatter...
114,05c80c6dafa54e2caa8a3b4c4376dda3,24 hour ban on yid run Facebook because of my ...
115,821474da654a451993ccadb5f2210271,#wp #wlw #wpww
116,19aebc19584e4fa5bfb82edb1d10b6ca,#Voting takes a plan:\n▫️Will you #vote on #El...
